In [95]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset,TensorDataset
from autoencoder import Autoencoder
from cifar_autoencoder import Cifar_Autoencoder
import torchvision
from model2 import classification_model
from model import cifar_classification_model
import copy
import partition
from pca import PCADigitReducer
from autoencoder import reduce_dimensions
from training import train,test
from federated_learning import distribute_global_model, federated_averaging

In [97]:
# Predefined stuff

n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10
num_clusters = 2

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

Mean: tensor([0.4914, 0.4822, 0.4465])
Std: tensor([0.2023, 0.1994, 0.2010])

In [99]:
cifar10_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Precomputed mean and std for CIFAR-10
])

cifar10_train_loader = DataLoader(
    datasets.CIFAR10('/files/', train=True, download=True, transform=cifar10_transform),
    batch_size=batch_size_train, shuffle=True
)

cifar10_test_loader = DataLoader(
    datasets.CIFAR10('/files/', train=False, download=True, transform=cifar10_transform),
    batch_size=batch_size_test, shuffle=True
)

Files already downloaded and verified
Files already downloaded and verified


In [100]:
for data, target in cifar10_train_loader:
    print(f"Data batch size: {data.size(0)}, Target batch size: {target.size(0)}")
    break

Data batch size: 64, Target batch size: 64


In [103]:
class CustomTensorDataset(TensorDataset):
    def __init__(self, *tensors):
        super().__init__(*tensors)
        self.data = tensors[0]
        self.targets = tensors[1]

In [105]:
train_loader_pca = copy.copy(cifar10_train_loader)
test_loader_pca = copy.copy(cifar10_test_loader)

train_loader_auto = copy.copy(cifar10_train_loader)
test_loader_auto = copy.copy(cifar10_test_loader)

In [107]:
print((cifar10_train_loader.dataset))

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /files/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )


In [109]:
train_data = []
train_labels = []
for data, labels in train_loader_pca:  # Use your CIFAR-10 DataLoader here
    train_data.append(data.view(data.size(0), -1))  # Flatten images
    train_labels.append(labels)
train_data = torch.cat(train_data, dim=0)  # Combine all batches
train_labels = torch.cat(train_labels, dim=0)

# Convert to numpy for PCA
train_data_np = train_data.numpy()

# Perform PCA
n_components = 100  # Set the desired number of components
pca = PCADigitReducer(n_components)
train_data_reduced = pca.fit_transform(train_data_np)  # Reduce dimensions

# Reconstruct the dataset from the reduced dimensions
train_data_reconstructed_np = pca.inverse_transform(train_data_reduced) 
train_data_reconstructed = torch.tensor(train_data_reconstructed_np, dtype=torch.float32)

# Reshape the reconstructed data back into the original image dimensions
train_data_reconstructed = train_data_reconstructed.view(-1, 3, 32, 32)

# Normalize the reconstructed dataset (use CIFAR-10 mean and std)
train_data_reconstructed = (train_data_reconstructed - torch.tensor([0.4914, 0.4822, 0.4465]).view(1, 3, 1, 1)) / \
                           torch.tensor([0.2470, 0.2435, 0.2616]).view(1, 3, 1, 1)

# Create a new DataLoader for the reconstructed data
batch_size_train = cifar10_train_loader.batch_size
train_dataset_pca = CustomTensorDataset(train_data_reconstructed, train_labels)
train_loader_reduced_pca = DataLoader(train_dataset_pca, batch_size=batch_size_train, shuffle=True)

In [110]:
trainingset_pca = train_loader_reduced_pca.dataset

print(trainingset_pca.targets)

tensor([7, 1, 1,  ..., 0, 5, 9])


In [111]:
latent_dim = 100  # Adjust latent dimension as needed
autoencoder = Cifar_Autoencoder(latent_dim=latent_dim)
auto_criterion = nn.MSELoss()
auto_optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)
auto_num_epochs = 5

for epoch in range(auto_num_epochs):
    for images, _ in cifar10_train_loader:  # Use your CIFAR-10 DataLoader here
        auto_optimizer.zero_grad()
        
        # Forward pass
        reconstructed = autoencoder(images)
        
        # Compute reconstruction loss
        loss = auto_criterion(reconstructed, images)
        
        # Backward pass and optimization
        loss.backward()
        auto_optimizer.step()
        
    print(f"Epoch [{epoch+1}/{auto_num_epochs}], Loss: {loss.item()}")


Epoch [1/5], Loss: 1.0315274000167847
Epoch [2/5], Loss: 0.8774186968803406
Epoch [3/5], Loss: 0.8388128280639648
Epoch [4/5], Loss: 0.9051497578620911
Epoch [5/5], Loss: 0.8178097605705261


In [112]:
autoencoder.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
latent_features, labels = reduce_dimensions(train_loader_auto, autoencoder.encoder, device)
latent_features = latent_features.detach()

reconstructed_images = autoencoder.decoder(latent_features.to(device))  
reconstructed_images = reconstructed_images.view(-1, 3, 32, 32) # Reshape to [batch_size, channels, height, width]

reconstructed_dataset = CustomTensorDataset(reconstructed_images.cpu(), labels)  
reduced_train_loader_auto = DataLoader(reconstructed_dataset, batch_size=batch_size_train, shuffle=True)

In [113]:
# Now partition them into 4 clients for federated learning
# pca 4 clients
trainingset_pca = train_loader_reduced_pca.dataset
partitioned_data_pca = partition.balanced_dirichlet_partition(trainingset_pca, partitions_number=4, alpha=0.5)
# Check sizes

for i in partitioned_data_pca:
    print(len(partitioned_data_pca[i]))

18580
9043
14723
7654


In [114]:
# Apply clustering
import cluster

cluster = cluster.Cluster(num_clusters=num_clusters)

targets = trainingset_pca.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_pca, targets, num_classes)

partitioned_data_pca = clustered_data

for key, value in partitioned_data_pca.items():
    print(f"Key: {key}, Length of Values: {len(value)}")


Key: 0, Length of Values: 26234
Key: 1, Length of Values: 23766


In [115]:
pca_client_loaders = [
    DataLoader(Subset(trainingset_pca, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_pca.values()
]

In [116]:
# check shape
x = pca_client_loaders[0].dataset
for i in range(9):
    sample, label = x[i]
    print(f"Sample {i} shape: {sample.shape}, Label: {label}")

Sample 0 shape: torch.Size([3, 32, 32]), Label: 8
Sample 1 shape: torch.Size([3, 32, 32]), Label: 1
Sample 2 shape: torch.Size([3, 32, 32]), Label: 2
Sample 3 shape: torch.Size([3, 32, 32]), Label: 5
Sample 4 shape: torch.Size([3, 32, 32]), Label: 1
Sample 5 shape: torch.Size([3, 32, 32]), Label: 2
Sample 6 shape: torch.Size([3, 32, 32]), Label: 1
Sample 7 shape: torch.Size([3, 32, 32]), Label: 8
Sample 8 shape: torch.Size([3, 32, 32]), Label: 8


In [117]:
# classic
trainingset = cifar10_train_loader.dataset
print(type(trainingset))  # Should be torchvision.datasets.CIFAR10
print(len(trainingset))  # Total number of samples in the training dataset
print(trainingset.targets[:10])  # First 10 targets
partitioned_data_classic = partition.balanced_dirichlet_partition(trainingset, partitions_number=4, alpha=0.5)

for i in partitioned_data_classic:
    print(len(partitioned_data_classic[i]))

<class 'torchvision.datasets.cifar.CIFAR10'>
50000
[6, 9, 9, 4, 1, 1, 2, 7, 8, 3]
18580
9043
14723
7654


In [118]:
# Apply clustering
import cluster

cluster = cluster.Cluster(num_clusters=num_clusters)

targets = trainingset.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_classic, targets, num_classes)

partitioned_data_classic = clustered_data

for key, value in partitioned_data_classic.items():
    print(f"Key: {key}, Length of Values: {len(value)}")


Key: 0, Length of Values: 26234
Key: 1, Length of Values: 23766


In [119]:
classic_client_loaders = [
    DataLoader(Subset(trainingset, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_classic.values()
]
for loader in classic_client_loaders:
    for data, target in loader:
        print(f"Data batch size: {data.size(0)}, Target batch size: {target.size(0)}")
        break

Data batch size: 64, Target batch size: 64
Data batch size: 64, Target batch size: 64


In [120]:
# auto 4 clients
trainingset_auto = reduced_train_loader_auto.dataset
print(trainingset_auto.targets)
print(trainingset_auto.data.shape)
partitioned_data_auto = partition.balanced_dirichlet_partition(trainingset_auto, partitions_number=4, alpha=0.5)
# Check sizes

for i in partitioned_data_auto:
    print(len(partitioned_data_auto[i]))

tensor([6, 6, 5,  ..., 9, 2, 6])
torch.Size([50000, 3, 32, 32])
18580
9043
14723
7654


In [121]:
auto_client_loaders = [
    DataLoader(Subset(trainingset_auto, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_auto.values()
]

In [122]:
# defining model for pca and autoencoder

global_model_pca = cifar_classification_model()
global_model_auto = cifar_classification_model()
gloabl_model_classic =cifar_classification_model()

num_clients = 2
# classic model
local_models_classic = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# pca models 
local_models_pca = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# autoencodere models
local_model_autoencoder = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]

# Classic

In [123]:
rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, classic_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        
    print(f"after training{local_models_classic}")
    global_weights_classic = federated_averaging(local_weights_classic)
    print(f"after fedaveraging{local_models_classic}")

    distribute_global_model(global_weights_classic,local_models_classic,single=False)

    distribute_global_model(global_weights_classic,gloabl_model_classic,single=True)
    test_losses = []
    test(gloabl_model_classic,cifar10_test_loader,test_losses)

Round 1/4
Training client 1


C:\Users\nazek\Documents\Federated-Dimensionality-Reduction\model.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/26234 (0%)]	Loss: 2.306071
Train Epoch: 1 [640/26234 (2%)]	Loss: 2.248354
Train Epoch: 1 [1280/26234 (5%)]	Loss: 2.190704
Train Epoch: 1 [1920/26234 (7%)]	Loss: 2.231462
Train Epoch: 1 [2560/26234 (10%)]	Loss: 2.223861
Train Epoch: 1 [3200/26234 (12%)]	Loss: 2.106492
Train Epoch: 1 [3840/26234 (15%)]	Loss: 2.140753
Train Epoch: 1 [4480/26234 (17%)]	Loss: 2.097965
Train Epoch: 1 [5120/26234 (20%)]	Loss: 2.051507
Train Epoch: 1 [5760/26234 (22%)]	Loss: 2.129118
Train Epoch: 1 [6400/26234 (24%)]	Loss: 2.153726
Train Epoch: 1 [7040/26234 (27%)]	Loss: 2.136460
Train Epoch: 1 [7680/26234 (29%)]	Loss: 2.060850
Train Epoch: 1 [8320/26234 (32%)]	Loss: 2.067832
Train Epoch: 1 [8960/26234 (34%)]	Loss: 2.060843
Train Epoch: 1 [9600/26234 (37%)]	Loss: 1.935715
Train Epoch: 1 [10240/26234 (39%)]	Loss: 2.062046
Train Epoch: 1 [10880/26234 (41%)]	Loss: 1.864512
Train Epoch: 1 [11520/26234 (44%)]	Loss: 1.910174
Train Epoch: 1 [12160/26234 (46%)]	Loss: 1.945527
Train Epoch: 1 [12800/26

C:\Users\nazek\anaconda3\Lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 1.8612, Accuracy: 3004/10000 (30%)

Round 2/4
Training client 1
Train Epoch: 1 [0/26234 (0%)]	Loss: 1.905541
Train Epoch: 1 [640/26234 (2%)]	Loss: 1.765914
Train Epoch: 1 [1280/26234 (5%)]	Loss: 1.823614
Train Epoch: 1 [1920/26234 (7%)]	Loss: 1.642334
Train Epoch: 1 [2560/26234 (10%)]	Loss: 1.742704
Train Epoch: 1 [3200/26234 (12%)]	Loss: 1.801540
Train Epoch: 1 [3840/26234 (15%)]	Loss: 1.601100
Train Epoch: 1 [4480/26234 (17%)]	Loss: 1.561047
Train Epoch: 1 [5120/26234 (20%)]	Loss: 1.675362
Train Epoch: 1 [5760/26234 (22%)]	Loss: 1.572510
Train Epoch: 1 [6400/26234 (24%)]	Loss: 1.874852
Train Epoch: 1 [7040/26234 (27%)]	Loss: 1.759430
Train Epoch: 1 [7680/26234 (29%)]	Loss: 1.395145
Train Epoch: 1 [8320/26234 (32%)]	Loss: 1.808410
Train Epoch: 1 [8960/26234 (34%)]	Loss: 1.611914
Train Epoch: 1 [9600/26234 (37%)]	Loss: 1.775683
Train Epoch: 1 [10240/26234 (39%)]	Loss: 1.537661
Train Epoch: 1 [10880/26234 (41%)]	Loss: 1.365071
Train Epoch: 1 [11520/26234 (44%)]	Los

# PCA

In [124]:
rounds_pca = 4

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, pca_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        
    print(f"after training{local_models_pca}")
    global_weights_pca = federated_averaging(local_weights_pca)
    print(f"after fedaveraging{local_models_pca}")

    distribute_global_model(global_weights_pca,local_models_pca,single=False)

    distribute_global_model(global_weights_pca,global_model_pca,single=True)
    test_losses = []
    test(global_model_pca,test_loader_pca,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/26234 (0%)]	Loss: 2.487489
Train Epoch: 1 [640/26234 (2%)]	Loss: 2.282407
Train Epoch: 1 [1280/26234 (5%)]	Loss: 2.195575
Train Epoch: 1 [1920/26234 (7%)]	Loss: 2.073159
Train Epoch: 1 [2560/26234 (10%)]	Loss: 2.060183
Train Epoch: 1 [3200/26234 (12%)]	Loss: 1.992629
Train Epoch: 1 [3840/26234 (15%)]	Loss: 2.102981
Train Epoch: 1 [4480/26234 (17%)]	Loss: 2.024119
Train Epoch: 1 [5120/26234 (20%)]	Loss: 2.112743
Train Epoch: 1 [5760/26234 (22%)]	Loss: 1.907293
Train Epoch: 1 [6400/26234 (24%)]	Loss: 1.794549
Train Epoch: 1 [7040/26234 (27%)]	Loss: 1.863352
Train Epoch: 1 [7680/26234 (29%)]	Loss: 1.746799
Train Epoch: 1 [8320/26234 (32%)]	Loss: 1.702721
Train Epoch: 1 [8960/26234 (34%)]	Loss: 1.917175
Train Epoch: 1 [9600/26234 (37%)]	Loss: 1.998555
Train Epoch: 1 [10240/26234 (39%)]	Loss: 1.745011
Train Epoch: 1 [10880/26234 (41%)]	Loss: 1.868625
Train Epoch: 1 [11520/26234 (44%)]	Loss: 1.830991
Train Epoch: 1 [12160/26234 (46%)]	Loss: 1.651

# Autoencoder

In [127]:
# Check shapes for autoencoder

data_test = auto_client_loaders[0].dataset
for i in range(len(data_test)):
    sample, label = data_test[i]
    print(f"Sample {i} shape: {sample.shape}, Label: {label}")

Sample 0 shape: torch.Size([3, 32, 32]), Label: 8
Sample 1 shape: torch.Size([3, 32, 32]), Label: 1
Sample 2 shape: torch.Size([3, 32, 32]), Label: 2
Sample 3 shape: torch.Size([3, 32, 32]), Label: 5
Sample 4 shape: torch.Size([3, 32, 32]), Label: 1
Sample 5 shape: torch.Size([3, 32, 32]), Label: 2
Sample 6 shape: torch.Size([3, 32, 32]), Label: 1
Sample 7 shape: torch.Size([3, 32, 32]), Label: 8
Sample 8 shape: torch.Size([3, 32, 32]), Label: 8
Sample 9 shape: torch.Size([3, 32, 32]), Label: 0
Sample 10 shape: torch.Size([3, 32, 32]), Label: 2
Sample 11 shape: torch.Size([3, 32, 32]), Label: 1
Sample 12 shape: torch.Size([3, 32, 32]), Label: 5
Sample 13 shape: torch.Size([3, 32, 32]), Label: 8
Sample 14 shape: torch.Size([3, 32, 32]), Label: 5
Sample 15 shape: torch.Size([3, 32, 32]), Label: 5
Sample 16 shape: torch.Size([3, 32, 32]), Label: 1
Sample 17 shape: torch.Size([3, 32, 32]), Label: 2
Sample 18 shape: torch.Size([3, 32, 32]), Label: 9
Sample 19 shape: torch.Size([3, 32, 32]),

In [128]:
rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, auto_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder,single=False)

    distribute_global_model(global_weights_auto, global_model_auto,single=True)
    test_losses = []
    test(global_model_auto,test_loader_auto,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/18580 (0%)]	Loss: 2.337794
Train Epoch: 1 [640/18580 (3%)]	Loss: 2.316845
Train Epoch: 1 [1280/18580 (7%)]	Loss: 2.265280
Train Epoch: 1 [1920/18580 (10%)]	Loss: 2.235583
Train Epoch: 1 [2560/18580 (14%)]	Loss: 2.231394
Train Epoch: 1 [3200/18580 (17%)]	Loss: 2.143690
Train Epoch: 1 [3840/18580 (21%)]	Loss: 2.043534
Train Epoch: 1 [4480/18580 (24%)]	Loss: 2.087844
Train Epoch: 1 [5120/18580 (27%)]	Loss: 2.008364
Train Epoch: 1 [5760/18580 (31%)]	Loss: 1.939745
Train Epoch: 1 [6400/18580 (34%)]	Loss: 2.005167
Train Epoch: 1 [7040/18580 (38%)]	Loss: 1.964993
Train Epoch: 1 [7680/18580 (41%)]	Loss: 2.021690
Train Epoch: 1 [8320/18580 (45%)]	Loss: 2.053171
Train Epoch: 1 [8960/18580 (48%)]	Loss: 2.063140
Train Epoch: 1 [9600/18580 (52%)]	Loss: 2.009491
Train Epoch: 1 [10240/18580 (55%)]	Loss: 1.993787
Train Epoch: 1 [10880/18580 (58%)]	Loss: 2.049904
Train Epoch: 1 [11520/18580 (62%)]	Loss: 1.897794
Train Epoch: 1 [12160/18580 (65%)]	Loss: 1.88